#  Section 1 : Python ETL

Section 1 Details: Database Configuration & Python ETL 
Provision one database of your choosing (SQL, NoSQL, Graph).  Write a python ETL that ingests the provided data, transforms it in some way, and loads it into the database. This should be reproducible code with documentation. (Terraform / Cloudformation / Ansible, docker-compose etc). 


## Source File Ingestion

In [1]:
import os
os.getcwd()


'/Users/sha/Desktop/Deloitte Assignment'

In [2]:
import time
start_time = time.time()

In [3]:
import pandas as pd

Source file is copied in the local and ingested into the python dataframe 'delodb':

In [4]:
delodb = pd.read_excel("/Users/sha/Desktop/Deloitte Assignment/DEM_Challenge_Section1_DATASET.xlsx")

Basic checks on source file:

In [5]:
delodb.head(5)

,id,first_name,last_name,email,gender,ip_address
0,1,Margaretta,Laughtisse,mlaughtisse0@mediafire.com,Genderfluid,34.148.232.131
1,2,Vally,Garment,vgarment1@wisc.edu,Bigender,15.158.123.36
2,3,Tessa,Curee,tcuree2@php.net,Bigender,132.209.143.225
3,4,Arman,Heineking,aheineking3@tuttocitta.it,Male,157.110.61.233
4,5,Roselia,Trustie,rtrustie4@ft.com,Non-binary,49.55.218.81


In [6]:
delodb.isnull().values.any()

False

Inference: No null values

In [7]:
print(delodb.dtypes)

id             int64
first_name    object
last_name     object
email         object
gender        object
ip_address    object
dtype: object


In [8]:
print(delodb.shape)

(1000, 6)


ETL Transformations: 

    1) Sorting the DF by name for better readability and adding a new column to store full name:

In [9]:
delodb = delodb.sort_values('first_name')

In [10]:
print(delodb.gender.unique())

['Genderqueer' 'Non-binary' 'Bigender' 'Agender' 'Male' 'Female'
 'Genderfluid' 'Polygender']


In [11]:
delodb['name'] = delodb['first_name'] + ' ' + delodb['last_name']

In [12]:
delodb.head(5)

,id,first_name,last_name,email,gender,ip_address,name
448,449,Abagael,Hurler,ahurlercg@tiny.cc,Genderqueer,41.194.203.182,Abagael Hurler
743,744,Abbey,Franc,afranckn@berkeley.edu,Non-binary,111.46.149.109,Abbey Franc
493,494,Abbie,Foch,afochdp@vistaprint.com,Bigender,236.68.102.75,Abbie Foch
262,263,Abbye,Prover,aprover7a@gravatar.com,Bigender,174.56.201.229,Abbye Prover
796,797,Abraham,Knutton,aknuttonm4@businessweek.com,Non-binary,191.166.25.16,Abraham Knutton


ETL Transformations: 

    1) Move the position of the new column next to name fields.  
    2) Reset the Index and resetting the id column.

In [13]:
delodb = delodb.iloc[:,[0,1,2,6,3,4,5]]

In [14]:
delodb = delodb.reset_index(drop=True)

In [15]:
delodb['id'] = delodb.index + 1

In [16]:
delodb.head(5)

,id,first_name,last_name,name,email,gender,ip_address
0,1,Abagael,Hurler,Abagael Hurler,ahurlercg@tiny.cc,Genderqueer,41.194.203.182
1,2,Abbey,Franc,Abbey Franc,afranckn@berkeley.edu,Non-binary,111.46.149.109
2,3,Abbie,Foch,Abbie Foch,afochdp@vistaprint.com,Bigender,236.68.102.75
3,4,Abbye,Prover,Abbye Prover,aprover7a@gravatar.com,Bigender,174.56.201.229
4,5,Abraham,Knutton,Abraham Knutton,aknuttonm4@businessweek.com,Non-binary,191.166.25.16


In [17]:
delodb['id'] = delodb['id'].apply(str)

ETL Transformation:

    1) Validate email field and add a new result column


In [18]:
from email_validator import validate_email, EmailNotValidError

def validate_e(x):
    try:
        v = validate_email(x)
        return True
    except EmailNotValidError as e:
        return False

delodb["Email_validate"] = delodb['email'].apply(validate_e)

In [19]:
delodb.head(5)

,id,first_name,last_name,name,email,gender,ip_address,Email_validate
0,1,Abagael,Hurler,Abagael Hurler,ahurlercg@tiny.cc,Genderqueer,41.194.203.182,True
1,2,Abbey,Franc,Abbey Franc,afranckn@berkeley.edu,Non-binary,111.46.149.109,True
2,3,Abbie,Foch,Abbie Foch,afochdp@vistaprint.com,Bigender,236.68.102.75,True
3,4,Abbye,Prover,Abbye Prover,aprover7a@gravatar.com,Bigender,174.56.201.229,True
4,5,Abraham,Knutton,Abraham Knutton,aknuttonm4@businessweek.com,Non-binary,191.166.25.16,True


In [20]:
delodb['Email_validate'].value_counts()

True     970
False     30
Name: Email_validate, dtype: int64

In [21]:
delodb.loc[delodb['Email_validate'] == False]

,id,first_name,last_name,name,email,gender,ip_address,Email_validate
14,15,Aguie,Jacobowicz,Aguie Jacobowicz,ajacobowicz81@blogspot.com,Bigender,64.183.48.133,False
47,48,Angelo,Bold,Angelo Bold,abolddt@blogspot.com,Non-binary,36.193.209.204,False
99,100,Becka,McGeraghty,Becka McGeraghty,bmcgeraghty65@geocities.jp,Genderqueer,201.155.43.236,False
107,108,Berke,Wadmore,Berke Wadmore,bwadmore47@blogspot.com,Male,195.193.157.134,False
127,128,Bourke,Garshore,Bourke Garshore,bgarshorepd@gov.uk,Genderqueer,242.87.150.115,False
178,179,Cassie,Perschke,Cassie Perschke,cperschkec@goo.gl,Genderqueer,193.62.46.4,False
193,194,Cherise,Pollock,Cherise Pollock,cpollock3j@storify.com,Agender,113.114.92.83,False
196,197,Chevalier,Cartmail,Chevalier Cartmail,ccartmailf1@flavors.me,Male,29.63.171.91,False
241,242,Danya,McGavigan,Danya McGavigan,dmcgavigandx@gov.uk,Agender,150.216.40.198,False
259,260,Debora,Coupar,Debora Coupar,dcoupar1e@senate.gov,Genderfluid,203.61.158.215,False


In [22]:
delodb['Email_validate'] = delodb['Email_validate'].apply(str)

## Checking Azure Connectivity

In [23]:
# Import the needed credential and management objects from the libraries.
from azure.identity import AzureCliCredential
from azure.mgmt.resource import ResourceManagementClient
import os

In [24]:
# Acquire a credential object using CLI-based authentication.
credential = AzureCliCredential()

In [25]:
# Retrieve subscription ID from environment variable.
subscription_id = "ee358da8-eab4-404b-817c-4c92df13d813"

In [26]:
# Obtain the management object for resources.
resource_client = ResourceManagementClient(credential, subscription_id)

In [27]:
# Retrieve the list of resource groups
group_list = resource_client.resource_groups.list()

In [28]:
# Show the groups in formatted output
column_width = 40

In [29]:
print("Resource Group".ljust(column_width) + "Location")
print("-" * (column_width * 2))
for group in list(group_list):
    print(f"{group.name:<{column_width}}{group.location}")

Resource Group                          Location
--------------------------------------------------------------------------------
rg_ml_dev                               eastus2
PythonAzureExample-DB-rg                eastus2
Deloitte-Assg1                          eastus2


Inference: Connection Success

## Load Data from DataFrame to Azure mySQL DB

In [30]:
import mysql.connector

In [31]:
db_server_name = "delo1server"
db_admin_name = "shasqladmin"
db_admin_password = "Passpass0"

db_name = "delo1db"
db_port = 3306

connection = mysql.connector.connect(user=f"{db_admin_name}@{db_server_name}",
    password=db_admin_password, host=f"{db_server_name}.mysql.database.azure.com",
    port=db_port, database=db_name, ssl_ca='./BaltimoreCyberTrustRoot.crt.pem')

cursor = connection.cursor()

print("Successfull Connection")

Successfull Connection


In [32]:
table_name = "EmployeeTable"

sql_create = "DROP TABLE IF EXISTS EmployeeTable"
cursor.execute(sql_create)

sql_create = f"CREATE TABLE {table_name} (id varchar(255),first_name varchar(255), last_name varchar(255),name varchar(255),email varchar(255),gender varchar(255), ip_address varchar(255), Email_validate varchar(255))"

cursor.execute(sql_create)
print(f"Successfully created table {table_name}")

Successfully created table EmployeeTable


In [33]:
for ind in delodb.index:

    sql_insert = "INSERT INTO EmployeeTable (id, first_name,last_name,name,email,gender,ip_address, Email_validate) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    
    val = (delodb['id'][ind], delodb['first_name'][ind], delodb['last_name'][ind], delodb['name'][ind], delodb['email'][ind], delodb['gender'][ind], delodb['ip_address'][ind], delodb['Email_validate'][ind])

    cursor.execute(sql_insert,val)
    
print("Successfully inserted data into table")

Successfully inserted data into table


In [34]:
sql_select_values= f"SELECT * FROM {table_name}"

cursor.execute(sql_select_values)
row = cursor.fetchone()

while row:
    print(str(row[0]) + " " + str(row[1]) + " " + str(row[2]) + " " + str(row[3]) + " " + str(row[4]) + " " + str(row[5]) + " " + str(row[6]) + " " + str(row[7]))
    row = cursor.fetchone()

connection.commit()

1 Abagael Hurler Abagael Hurler ahurlercg@tiny.cc Genderqueer 41.194.203.182 True
2 Abbey Franc Abbey Franc afranckn@berkeley.edu Non-binary 111.46.149.109 True
3 Abbie Foch Abbie Foch afochdp@vistaprint.com Bigender 236.68.102.75 True
4 Abbye Prover Abbye Prover aprover7a@gravatar.com Bigender 174.56.201.229 True
5 Abraham Knutton Abraham Knutton aknuttonm4@businessweek.com Non-binary 191.166.25.16 True
6 Ad Cumber Ad Cumber acumberbu@utexas.edu Agender 254.164.242.220 True
7 Adelaide Yushachkov Adelaide Yushachkov ayushachkovdw@com.com Bigender 241.237.36.186 True
8 Adelbert Gerg Adelbert Gerg agergew@statcounter.com Male 31.199.211.165 True
9 Adolpho Toal Adolpho Toal atoal1d@intel.com Non-binary 96.121.42.172 True
10 Adolpho Dorro Adolpho Dorro adorro46@usda.gov Female 84.121.118.178 True
11 Adore Gallie Adore Gallie agalliehm@smugmug.com Genderfluid 162.126.224.49 True
12 Adrian MacCosto Adrian MacCosto amaccostonu@webmd.com Genderqueer 183.4.92.221 True
13 Aggy Ferrarin Aggy Ferr

Data loaded into table and extracted:

Program End

In [35]:
end_time = time.time()

In [36]:
Program_run_time = end_time - start_time

In [37]:
print("Runtime in minutes", Program_run_time/60)

Runtime in minutes 6.9302813649177555
